In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
df = pd.read_csv("../data/lifestyle_disease_transformed.csv")
df = df.drop(["Unnamed: 0"], axis=1)

target = df.iloc[:,1].values
features = df.iloc[:,2:-1].values

le = LabelEncoder()
ss = StandardScaler()

target = le.fit_transform(target).reshape(-1,1)
features = ss.fit_transform(features)

target = df.iloc[:,-1:].values
features = df.iloc[:,:-1].values

x_train = features[:1000]
y_train = target[:1000]
x_test = features[1000:1500]
y_test = target[1000:1500]

In [3]:
import heapq

class Classifier:
    def __init__(self, n_neighbors=3):
        self.n_neighbors = n_neighbors
    
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
    
    def distance(self, x, y):
        return np.sum((x - y) ** 2)
    
    def predict_single(self, x):
        heap = []
        
        for i in range(len(self.x_train)):
            dist = self.distance(self.x_train[i], x)

            heapq.heappush(heap, (-dist, self.y_train[i]))

            if len(heap) > self.n_neighbors:
                heapq.heappop(heap) 
        
        labels = [int(label) for (_, label) in heap]
        
        count = np.bincount(labels)
        return np.argmax(count)
    
    def predict(self, x_test):
        return np.array([self.predict_single(x) for x in x_test]).reshape(-1, 1)


In [5]:
model = Classifier(n_neighbors=3)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test) * 100)

C:\Users\JATIN\AppData\Local\Temp\ipykernel_20568\943071037.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels = [int(label) for (_, label) in heap]


88.2


In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc
sc = roc_auc_score(y_test,y_pred)
print("AUC_ROC: ",sc)
import matplotlib.pyplot as plt
p,q,r = roc_curve(y_test,y_pred)
plt.plot(p,q)
plt.xlabel("fpr")
plt.ylabel("tpr")
plt.title("Roc_curve")
plt.show()

AUC_ROC:  0.8792030001274326
